# Capstone Project- Data Science


In [17]:
import pandas as pd
import numpy as np
import requests
# !pip install wikipedia
# import wikipedia as wiki
# !pip install geopy.extra
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

In [18]:
print('Hello Capstone project Course')

Hello Capstone project Course


# TriCity Poland

My project looks at grouping similar living areas across the various districts in the TriCity.

In [19]:
# District names take from Wikipideda

gdansk_districts=['Osowa','Oliwa','Żabianka, Jelitkowo','Przymorze Małe','Przymorze Wielkie','VII Dwór','Strzyża',
                  'Zaspa-Młyniec','Zaspa-Rozstaje','Brzeźno','Matarnia','Brętowo','Wrzeszcz Dolny','Wrzeszcz Górny',
                  'Letnica','Nowy Port','Piecki-Migowo','Suchanino','Siedlce','Wzgórze Mickiewicza','Aniołki','Młyniska',
                  'Stogi','Przeróbką','Śródmieście','Krakowiec-Górki Zachodnie','Wyspa Sobieszewska','Kokoszki','Chełm',
                  'Ujeścisko-Łostowice','Jasień','Orunia-Św. Wojciech-Lipce','Olszynka','Rudniki']

sopot_districts=['Sopot Centrum','Karlikowo','Kamienny Potok','Przylesie','Dolny Sopot Grunwaldzka','Dolny Sopot Haffnera',
                 'Lisie Wzgórze','Zajęcze Wzgórze','Górny Sopot','Brodwino','Osiedle Mickiewicza','Świemirowo','Stawowie',
                 'Leśna Polana\Gręzowo','Centrum Południe – Kościuszki','Sopocki Las']

gdynia_districts=['Babie Doły', 'Chwarzno-Wiczlino', 'Chylonia', 'Cisowa', 'Dąbrowa', 'Działki Leśne', 'Grabówek', 
                  'Kamienna Góra', 'Karwiny', 'Leszczynki', 'Mały Kack', 'Obłuże', 'Oksywie', 'Orłowo', 'Pogórze', 'Pustki Cisowskie-Demptowo', 
                  'Redłowo', 'Śródmieście', 'Wielki Kack', 'Witomino-Leśniczówka', 'Witomino-Radiostacja', 'Wzgórze Św. Maksymiliana']

df=pd.DataFrame()

df_gdansk=pd.DataFrame(columns=['District'])
df_sopot=pd.DataFrame(columns=['District'])
df_gdynia=pd.DataFrame(columns=['District'])

In [20]:
df_gdansk['District']=gdansk_districts
df_sopot['District']=sopot_districts
df_gdynia['District']=gdynia_districts

In [21]:
df_sopot.head()

,District
0,Sopot Centrum
1,Karlikowo
2,Kamienny Potok
3,Przylesie
4,Dolny Sopot Grunwaldzka


In [22]:
df=df.append(sopot_districts)
df=df.append(gdansk_districts)
df=df.append(gdynia_districts)

df.rename(columns={0:'Districts'},inplace=True)    

df.reset_index(drop=True,inplace=True)



In [23]:
df['Latitude']='default'
df['Longitude']='default'

In [30]:
for i in range(len(df['Districts'])):
    address=df['Districts'][i]+', Poland'
    location=None
    while location is None :
        geolocator=Nominatim(user_agent='Capstone')
        location=geolocator.geocode(address)
    df['Latitude'][i]=location.latitude
    df['Longitude'][i]=location.longitude

GeocoderServiceError: HTTP Error 429: Too Many Requests

In [31]:
df.head(40)

,Districts,Latitude,Longitude
0,Sopot Centrum,default,default
1,Karlikowo,default,default
2,Kamienny Potok,default,default
3,Przylesie,default,default
4,Dolny Sopot Grunwaldzka,default,default
5,Dolny Sopot Haffnera,default,default
6,Lisie Wzgórze,default,default
7,Zajęcze Wzgórze,default,default
8,Górny Sopot,default,default
9,Brodwino,default,default
